# one version

In [1]:
%run utils.ipynb 

2024-11-05 20:13:17.744059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data = load_data()

## EDA

In [23]:
# Load your dataset
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss, ccf
import matplotlib.pyplot as plt

In [4]:
res = sm.tsa.seasonal_decompose(data['price'], model='additive')

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(20, 12))
res.observed.plot(ax=ax1, title='Observed')
res.trend.plot(ax=ax2, title='Trend')
res.resid.plot(ax=ax3, title='Residual')
res.seasonal.plot(ax=ax4, title='Seasonal')
plt.tight_layout()
plt.show()

The Augmented Dickey-Fuller (ADF) test, a type of unit root test, determines how strongly a time series is defined by a trend. Its hypotheses are the following:

- Null Hypothesis  𝐻0
 : There is a unit root in the time series, i.e. the series is autocorrelated with (r=1), a time dependent structure and thus, is not stationary.

- Alternate Hypothesis  𝐻1
 : The time series has no unit root and is either stationary or can be made stationary using differencing.

In [5]:
y = data['price']
adf_test = adfuller(y, regression='c')
print('ADF Statistic: {:.6f}\np-value: {:.6f}\n#Lags used: {}'
      .format(adf_test[0], adf_test[1], adf_test[2]))
for key, value in adf_test[4].items():
    print('Critical Value ({}): {:.6f}'.format(key, value))

ADF Statistic: -6.460620
p-value: 0.000000
#Lags used: 44
Critical Value (1%): -3.430724
Critical Value (5%): -2.861705
Critical Value (10%): -2.566858


The ADF statistic (-6.461) is less than the critical value at 1% (-3.431) and thus, we can say that we reject the null hypothesis  𝐻0
  with a significance level 1%, meaning that there is not a root-unit in the time series and thus, that it is either stationary or could be made stationary with 1st order differencing (difference-stationary).

## Train data

In [6]:
train_data, test_data = train_test_split_7(data)

Shape of train_data: (13487, 5)
Shape of test_data: (4033, 5)


In [7]:
# Assuming train_data and test_data are pandas Series or DataFrame with datetime index
plt.figure(figsize=(32, 8))  # Set the size of the plot
plt.plot(train_data['price'], label='Training set')  # Plot training data
plt.plot(test_data['price'], color='r', label='Test set')  # Plot test data in red

plt.legend()  # Show legend to identify the datasets
plt.title('Day-ahead Electricity Price')  # Set the title of the plot
plt.show()  # Display the 

In [6]:
data['DayOfWeek'] = data.index.dayofweek              # Day of the week (0=Monday, 6=Sunday)
data['WeekOfYear'] = data.index.isocalendar().week    # ISO week number of the year
data['Day'] = data.index.day                          # Day of the month
data['Month'] = data.index.month                      # Month of the year
data['Year'] = data.index.year                        # Year
data['PeriodOfDay'] = data.index.hour                 # Hour of the day (0-23)

In [9]:
import holidays


# Initialize the Netherlands holidays for 2022 and 2023
# nl_holidays = holidays.Netherlands(years=[2022, 2023])

# Add a holiday flag to the dataset
# data['HolidayFlag'] = data.index.to_series().apply(lambda x: 1 if x in nl_holidays else 0)



In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17520 entries, 2022-01-01 01:00:00 to 2024-01-01 00:00:00
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   solar          17520 non-null  int64  
 1   wind_offshore  17520 non-null  int64  
 2   wind_onshore   17520 non-null  int64  
 3   total_load     17520 non-null  int64  
 4   price          17520 non-null  float64
 5   DayOfWeek      17520 non-null  int32  
 6   WeekOfYear     17520 non-null  UInt32 
 7   Day            17520 non-null  int32  
 8   Month          17520 non-null  int32  
 9   Year           17520 non-null  int32  
 10  PeriodOfDay    17520 non-null  int32  
dtypes: UInt32(1), float64(1), int32(5), int64(4)
memory usage: 1.2 MB


In [14]:
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']
features_2 = ['price', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']

In [ ]:
def model_runner(data, features):
    data = data[features]
    
    scaler = MinMaxScaler()
    data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
    train_data, test_data = train_test_split_7(data_scaled)
    
    seq_length = 24
    X_train_b, y_train_b = create_sequences(train_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)

    
    input_shape = (X_train_b.shape[1], X_train_b.shape[2])
    
    lstm_model_b = create_lstm_model(input_shape)
    lstm_model_b.fit(X_train_b, y_train_b, epochs=10, batch_size=32, validation_split=0.1)

    def smape(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    # Invert scaling for predictions and true values if scaling was applied
    lstm_preds_b = lstm_model_b.predict(X_test)
    lstm_preds_b_inverse = scaler.inverse_transform(
        np.concatenate((lstm_preds_b, X_test[:, -1, 1:]), axis=1))[:, 0]
    y_test_inverse = scaler.inverse_transform(
        np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
    
    # Calculate MAE and RMSE
    mae = mean_absolute_error(y_test_inverse, lstm_preds_b_inverse)
    rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_preds_b_inverse))
    smape_value = smape(y_test_inverse, lstm_preds_b_inverse)
    
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")

    return y_test_inverse, lstm_preds_b_inverse

features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']

y_ture, pred_e = model_runner(data,features)

In [28]:
pred_g = model_runner(data,features_2)

Shape of train_data: (13487, 7)
Shape of test_data: (4033, 7)
Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1111 - val_loss: 0.0425
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0583 - val_loss: 0.0376
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0518 - val_loss: 0.0355
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0459 - val_loss: 0.0270
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0391 - val_loss: 0.0269
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0333 - val_loss: 0.0256
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0312 - val_loss: 0.0254
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0287 - val_loss: 0.0243
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0279 - val_loss: 0.0219
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0270 - val_loss: 0.0224
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Mean Absolute Error (MAE): 28.00
Root Mean Squared Error (RMSE): 38.94
Symmetric Mean Absolute Percentage Err

In [30]:
y_ture, pred_e = model_runner(data,features)

Shape of train_data: (13487, 11)
Shape of test_data: (4033, 11)
Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1399 - val_loss: 0.0406
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0583 - val_loss: 0.0345
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0491 - val_loss: 0.0308
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0428 - val_loss: 0.0295
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0381 - val_loss: 0.0284
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0346 - val_loss: 0.0247
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0322 - val_loss: 0.0214
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0301 - val_loss: 0.0242
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0283 - val_loss: 0.0217
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0277 - val_loss: 0.0213
126/126 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Mean Absolute Error (MAE): 28.21
Root Mean Squared Error (RMSE): 39.21
Symmetric Mean Absolute Percentage Err

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features in X
        y.append(data.iloc[i+seq_length]['price'])  # Target is 'price'
    return np.array(X), np.array(y)

# Define an enhanced LSTM model with added layers and dropout
def create_enhanced_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(32, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# Main model runner function with enhanced LSTM
def model_runner(data, features):
    # Filter the dataset to include only the specified features
    data = data[features]
    
    # Scale the data
    scaler = MinMaxScaler()
    data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
    
    # Split the data into training and test sets
    train_data, test_data = train_test_split_7(data_scaled)  # Assuming train_test_split_7 is predefined

    # Set sequence length
    seq_length = 24
    X_train, y_train = create_sequences(train_data, seq_length)
    X_test, y_test = create_sequences(test_data, seq_length)
    
    # Define input shape for the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    # Initialize and train the enhanced LSTM model
    lstm_model = create_enhanced_lstm_model(input_shape)
    lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

    # Define sMAPE function for performance evaluation
    def smape(y_true, y_pred):
        return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    # Make predictions on the test set
    lstm_preds = lstm_model.predict(X_test)
    
    # Invert scaling for predictions and test values
    lstm_preds_inverse = scaler.inverse_transform(
        np.concatenate((lstm_preds, X_test[:, -1, 1:]), axis=1))[:, 0]
    y_test_inverse = scaler.inverse_transform(
        np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
    
    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test_inverse, lstm_preds_inverse)
    rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_preds_inverse))
    smape_value = smape(y_test_inverse, lstm_preds_inverse)
    
    # Print performance metrics
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")

    return y_test_inverse, lstm_preds_inverse

# Define features including energy generation features
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']

# Run the model with enhanced settings
y_true, pred_e = model_runner(data, features)

Shape of train_data: (13487, 11)
Shape of test_data: (4033, 11)
Epoch 1/20


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.1058 - val_loss: 0.0526
Epoch 2/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0579 - val_loss: 0.0386
Epoch 3/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0468 - val_loss: 0.0357
Epoch 4/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0402 - val_loss: 0.0349
Epoch 5/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0367 - val_loss: 0.0302
Epoch 6/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0361 - val_loss: 0.0361
Epoch 7/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.0335 - val_loss: 0.0299
Epoch 8/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0317 - val_loss: 0.0310
Epoch 9/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - loss: 0.0308 - val_loss: 0.0267
Epoch 10/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0295 - val_loss: 0.0254
Epoch 11/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0299 - val_loss: 0.0286
Epoch 12/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/ste

In [33]:

# Define features including energy generation features
features_b = ['price', 'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']

# Run the model with enhanced settings
y_true_b, pred_e_b = model_runner(data, features_b)

Shape of train_data: (13487, 7)
Shape of test_data: (4033, 7)
Epoch 1/20


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - loss: 0.1244 - val_loss: 0.0535
Epoch 2/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 0.0559 - val_loss: 0.0503
Epoch 3/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0517 - val_loss: 0.0555
Epoch 4/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0461 - val_loss: 0.0448
Epoch 5/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0425 - val_loss: 0.0350
Epoch 6/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.0393 - val_loss: 0.0326
Epoch 7/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0372 - val_loss: 0.0378
Epoch 8/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0361 - val_loss: 0.0419
Epoch 9/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0346 - val_loss: 0.0318
Epoch 10/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - loss: 0.0332 - val_loss: 0.0311
Epoch 11/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - loss: 0.0323 - val_loss: 0.0367
Epoch 12/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/s

In [15]:
data = data[features_2]

In [16]:
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
train_data, test_data = train_test_split_7(data_scaled)

seq_length = 24
X_train_b, y_train_b = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

Shape of train_data: (13487, 7)
Shape of test_data: (4033, 7)


In [17]:
X_test.shape

(4009, 24, 7)

In [18]:
input_shape = (X_train_b.shape[1], X_train_b.shape[2])

In [19]:
lstm_model_b = create_lstm_model(input_shape)
lstm_model_b.fit(X_train_b, y_train_b, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.1205 - val_loss: 0.0415
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0594 - val_loss: 0.0381
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0528 - val_loss: 0.0439
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0469 - val_loss: 0.0326
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0429 - val_loss: 0.0327
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0381 - val_loss: 0.0254
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0349 - val_loss: 0.0241
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.0314 - val_loss: 0.0217
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0302 - val_loss: 0.0201
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0285 - val_loss: 0.0294


In [20]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Invert scaling for predictions and true values if scaling was applied
lstm_preds_b = lstm_model_b.predict(X_test)
lstm_preds_b_inverse = scaler.inverse_transform(
    np.concatenate((lstm_preds_b, X_test[:, -1, 1:]), axis=1))[:, 0]
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]

# Calculate MAE and RMSE
mae = mean_absolute_error(y_test_inverse, lstm_preds_b_inverse)
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_preds_b_inverse))
smape_value = smape(y_test_inverse, lstm_preds_b_inverse)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")

126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Mean Absolute Error (MAE): 31.38
Root Mean Squared Error (RMSE): 42.58
Symmetric Mean Absolute Percentage Error (sMAPE): 21.84


In [16]:
lstm_preds_b[0:5]

array([[0.40596506],
       [0.3996216 ],
       [0.39575702],
       [0.39514902],
       [0.39580727]], dtype=float32)

In [17]:
X_test[:, -1, 1:]

array([[0.09531697, 0.02932586, 0.64462412, ..., 0.        , 0.        ,
        1.        ],
       [0.11700045, 0.04525451, 0.62037973, ..., 0.        , 0.        ,
        0.        ],
       [0.08071074, 0.03072789, 0.59078111, ..., 0.        , 0.        ,
        0.04347826],
       ...,
       [0.99277217, 0.64178058, 0.55912504, ..., 1.        , 0.5       ,
        0.91304348],
       [0.99337449, 0.64762239, 0.54003032, ..., 1.        , 0.5       ,
        0.95652174],
       [0.99292275, 0.65268528, 0.52399172, ..., 1.        , 0.5       ,
        1.        ]])

In [18]:
lstm_preds_b[:5]

array([[0.40596506],
       [0.3996216 ],
       [0.39575702],
       [0.39514902],
       [0.39580727]], dtype=float32)

In [19]:
lstm_preds_b_inverse = scaler.inverse_transform(
    np.concatenate((lstm_preds_b, X_test[:, -1, 1:]), axis=1))[:, 0]

In [20]:
lstm_preds_b_inverse[:5]

array([12701.02286702, 12502.56156135, 12381.6541127 , 12362.63231295,
       12383.22613144])

In [21]:
y_test_inverse[:5]

array([12228.71237287, 11951.1512768 , 12128.56146192, 11974.04291359,
       12128.56146192])

## debug

In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

2024-11-04 15:35:46.293955: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
data = pd.read_pickle('../data/data.pkl') 
data_im = data.copy()

In [5]:
data_im['DayOfWeek'] = data.index.dayofweek              # Day of the week (0=Monday, 6=Sunday)
data_im['WeekOfYear'] = data.index.isocalendar().week    # ISO week number of the year
data_im['Day'] = data.index.day                          # Day of the month
data_im['Month'] = data.index.month                      # Month of the year
data_im['Year'] = data.index.year                        # Year
data_im['PeriodOfDay'] = data.index.hour                 # Hour of the day (0-23)

NameError: name 'data_im' is not defined

In [5]:
def create_sequences(data, seq_length=24, target_column='price'):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features in X
        y.append(data[target_column].iloc[i+seq_length])  # Target is still the original 'price'
    return np.array(X), np.array(y)

def train_test_split(data):
    test_indices = data.index.to_series().groupby([data.index.year, data.index.month]).apply(lambda x: x[-24*7:])
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    return train_data, test_data

def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, activation='relu', input_shape=input_shape),
        Dropout(0.2),  # Dropout to prevent overfitting
        Dense(1)  # Output layer with a single neuron for regression
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

In [19]:
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']
features_2 = ['price', 
            'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']

In [21]:
data_im = data_im[features]

In [22]:
# from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_im), columns=data_im.columns, index=data_im.index)

In [23]:
train_data, test_data = train_test_split(data_scaled)

In [24]:
seq_length = 24
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [25]:
# def create_lstm_model(input_shape):
#     model = Sequential([
#         LSTM(50, activation='relu', input_shape=input_shape),
#         Dropout(0.2),  # Regularization to prevent overfitting
#         Dense(1)  # Output layer for regression
#     ])
#     model.compile(optimizer='adam', loss='mae')
#     return model

# Set input shape for the LSTM
input_shape = (X_train.shape[1], X_train.shape[2])

In [26]:
lstm_model = create_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1108 - val_loss: 0.0423
Epoch 2/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0552 - val_loss: 0.0304
Epoch 3/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0461 - val_loss: 0.0326
Epoch 4/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0402 - val_loss: 0.0275
Epoch 5/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0370 - val_loss: 0.0269
Epoch 6/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0334 - val_loss: 0.0230
Epoch 7/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0310 - val_loss: 0.0251
Epoch 8/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0298 - val_loss: 0.0210
Epoch 9/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0281 - val_loss: 0.0219
Epoch 10/10
379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0264 - val_loss: 0.0195


In [27]:
lstm_preds = lstm_model.predict(X_test)

# Invert scaling (if you scaled the data initially)
# In this case, we need to transform both predictions and true values back to original scale
lstm_preds_inverse = scaler.inverse_transform(
    np.concatenate((lstm_preds, X_test[:, -1, 1:]), axis=1))[:, 0]
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]


126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [28]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [29]:
lstm_smape = smape(y_test_inverse, lstm_preds_inverse)
print(f"LSTM sMAPE: {lstm_smape:.2f}")

# Calculate MAE
mae = mean_absolute_error(y_test_inverse, lstm_preds_inverse)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_preds_inverse))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

LSTM sMAPE: 18.81
Mean Absolute Error (MAE): 27.58
Root Mean Squared Error (RMSE): 38.86


In [41]:
# Create a time index for the test set based on the original data index
test_index = test_data.index[24:]  # Adjust for sequence length

# Plot the predictions and actual values
plt.figure(figsize=(34, 7))
plt.plot(test_index, y_test_inverse, label="Actual Values", color="blue")
plt.plot(test_index, lstm_preds_inverse, label="Predicted Values", color="red")
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("LSTM Model - Actual vs Predicted Values")
plt.legend()9
plt.show()

In [30]:
Y = lstm_preds_inverse

In [20]:
G = lstm_preds_inverse

In [ ]:
# add g
# LSTM sMAPE: 18.86
# Mean Absolute Error (MAE): 26.68
# Root Mean Squared Error (RMSE): 36.50

In [32]:
from dieboldmariano import dm_test

T = y_test_inverse
F = Y
G = G

dm_test(T, F, G, one_sided=True)

(4.81850134041317, 0.9999992500345725)

## EWT

In [37]:
data.columns

Index(['solar', 'wind_offshore', 'wind_onshore', 'total_load', 'price',
       'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay'],
      dtype='object')

In [30]:
import pandas as pd
import numpy as np
import pywt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load your dataset
data = data[data.columns]

# Define the features to decompose
features_to_decompose = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']

# Function to apply wavelet decomposition to each feature
def wavelet_decompose(series, wavelet='db1', level=3):
    coeffs = pywt.wavedec(series, wavelet, level=level)  # Decompose signal into frequency components
    decomposed_data = pd.DataFrame(coeffs).T  # Convert to DataFrame
    decomposed_data.columns = [f'{series.name}_wavelet_{i}' for i in range(len(coeffs))]
    return decomposed_data

# Apply wavelet decomposition to each feature and store the decomposed components
decomposed_features = []
for feature in features_to_decompose:
    decomposed = wavelet_decompose(data[feature], wavelet='db1', level=3)
    decomposed.index = data.index[:len(decomposed)]  # Align indices with the original data
    decomposed_features.append(decomposed)

# Concatenate decomposed features with the original data
decomposed_data = pd.concat(decomposed_features, axis=1)
data_with_wavelets = pd.concat([data, decomposed_data], axis=1).dropna()

# Scale the entire dataset (including decomposed features)
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_with_wavelets), columns=data_with_wavelets.columns, index=data_with_wavelets.index)

# Define a function to create sequences
def create_sequences(data, seq_length, target_column='price'):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Use all features within the sequence length
        y.append(data.iloc[i+seq_length][target_column])  # Target value at the next time step
    return np.array(X), np.array(y)

# Split data into train and test sets (e.g., 80% train, 20% test)
# train_size = int(len(data_scaled) * 0.8)
# train_data = data_scaled[:train_size]
# test_data = data_scaled[train_size:]

train_data, test_data = train_test_split_7(data_scaled)  # Assuming train_test_split_7 is predefined
# Create sequences for training and testing
seq_length = 24  # Define the sequence length
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Define the LSTM model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(32, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

# Define input shape and initialize the model
input_shape = (X_train.shape[1], X_train.shape[2])
lstm_model = create_lstm_model(input_shape)

# Train the model
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# Define sMAPE function for evaluation
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# Make predictions on the test set
lstm_preds = lstm_model.predict(X_test)

# Invert scaling for predictions and true values
lstm_preds_inverse = scaler.inverse_transform(
    np.concatenate((lstm_preds, X_test[:, -1, 1:]), axis=1))[:, 0]
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]

# Calculate MAE, RMSE, and sMAPE
mae = mean_absolute_error(y_test_inverse, lstm_preds_inverse)
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_preds_inverse))
smape_value = smape(y_test_inverse, lstm_preds_inverse)

# Print performance metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value:.2f}")


Shape of train_data: (1655, 33)
Shape of test_data: (535, 33)
Epoch 1/20


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.1999 - val_loss: 0.0826
Epoch 2/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0824 - val_loss: 0.1082
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0704 - val_loss: 0.1029
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0631 - val_loss: 0.0942
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0641 - val_loss: 0.0644
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0566 - val_loss: 0.0855
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0545 - val_loss: 0.1061
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0565 - val_loss: 0.0722
Epoch 9/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0569 - val_loss: 0.0627
Epoch 10/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0526 - val_loss: 0.0749
Epoch 11/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0508 - val_loss: 0.0862
Epoch 12/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0520 - val_l

## Attention

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import LSTM, Dense, Flatten, Dropout, Input, Multiply, Permute, RepeatVector
from keras.layers import Activation, Lambda
import keras.backend as K
from sklearn.metrics import mean_absolute_error, mean_squared_error
# from keras.layers import Dense, LSTM, GRU, Conv1D, 

# Load your dataset (assuming it has been preprocessed)

# Define features and target
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']
data = data[features]
target_column = 'price'

# Split into training and test sets
# train_size = int(len(data) * 0.8)
# train_data, test_data = data[:train_size], data[train_size:]

train_data, test_data = train_test_split_7(data)  # Assuming train_test_split_7 is predefined


# Scale the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

# Function to create sequences
seq_length = 24  # Define the sequence length

def create_sequences(data, seq_length, target_column_idx):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, target_column_idx])
    return np.array(X), np.array(y)

target_column_idx = data.columns.get_loc(target_column)  # Index of target column
X_train, y_train = create_sequences(train_scaled, seq_length, target_column_idx)
X_test, y_test = create_sequences(test_scaled, seq_length, target_column_idx)

# Attention layer
def attention_3d_block(inputs):
    # Compute the attention weights
    attention = Dense(1, activation='tanh')(inputs)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(inputs.shape[2])(attention)
    attention = Permute([2, 1])(attention)

    # Multiply the attention weights by the inputs
    output_attention = Multiply()([inputs, attention])
    return output_attention

# Define LSTM model with attention
def create_lstm_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(inputs)
    lstm_out = Dropout(0.2)(lstm_out)
    
    # Apply attention
    attention_out = attention_3d_block(lstm_out)
    attention_out = LSTM(32, return_sequences=False)(attention_out)
    attention_out = Dropout(0.2)(attention_out)
    
    outputs = Dense(1)(attention_out)  # Output layer
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='mae')
    return model

# Initialize and train the model
input_shape = (X_train.shape[1], X_train.shape[2])
lstm_attention_model = create_lstm_attention_model(input_shape)
lstm_attention_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Predict on test data
lstm_attention_preds = lstm_attention_model.predict(X_test)

# Invert scaling for predictions and actual values
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
lstm_attention_preds_inverse = scaler.inverse_transform(
    np.concatenate((lstm_attention_preds, X_test[:, -1, 1:]), axis=1))[:, 0]

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
# Calculate performance metrics
mae = mean_absolute_error(y_test_inverse, lstm_attention_preds_inverse)
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_attention_preds_inverse))

smape_value_attention = smape(y_test_inverse, lstm_attention_preds_inverse)

# Print performance metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value_attention:.2f}")

Shape of train_data: (13487, 11)
Shape of test_data: (4033, 11)
Epoch 1/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.1295 - val_loss: 0.0598
Epoch 2/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0620 - val_loss: 0.0494
Epoch 3/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0530 - val_loss: 0.0349
Epoch 4/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0468 - val_loss: 0.0370
Epoch 5/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0443 - val_loss: 0.0632
Epoch 6/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0396 - val_loss: 0.0355
Epoch 7/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0370 - val_loss: 0.0396
Epoch 8/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0344 - val_loss: 0.0283
Epoch 9/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0322 - val_loss: 0.0247
Epoch 10/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0303 - val_loss: 0.0252
Epoch 11/20
379/379 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0

In [11]:

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
# smape_value = smape(y_test_inverse, lstm_preds_inverse)
smape_value_attention = smape(y_test_inverse, lstm_attention_preds_inverse)

# Print performance metrics
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value_attention:.2f}")

Symmetric Mean Absolute Percentage Error (sMAPE): 18.07


## Lagged data

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Load and preprocess your dataset
# data = pd.read_csv('your_data.csv', parse_dates=['date'], index_col='date')
# data = load_data()
data = load_data()
data['DayOfWeek'] = data.index.dayofweek              # Day of the week (0=Monday, 6=Sunday)
data['WeekOfYear'] = data.index.isocalendar().week    # ISO week number of the year
data['Day'] = data.index.day                          # Day of the month
data['Month'] = data.index.month                      # Month of the year
data['Year'] = data.index.year                        # Year
data['PeriodOfDay'] = data.index.hour                 # Hour of the day (0-23)
# features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 'DayOfWeek', 'WeekOfYear', 'Day', 'Month', 'Year', 'PeriodOfDay']
# data = data[features]
# Add cyclic transformations for HourOfDay and DayOfWeek
data['HourOfDay_sin'] = np.sin(2 * np.pi * data['PeriodOfDay'] / 24)
data['HourOfDay_cos'] = np.cos(2 * np.pi * data['PeriodOfDay'] / 24)

data['DayOfWeek_sin'] = np.sin(2 * np.pi * data['DayOfWeek'] / 7)
data['DayOfWeek_cos'] = np.cos(2 * np.pi * data['DayOfWeek'] / 7)

# Drop original cyclic features if you want to avoid redundancy
data.drop(['PeriodOfDay', 'DayOfWeek'], axis=1, inplace=True)

# Use only current values for 'WeekOfYear', 'Day', 'Month', 'Year' without lagging
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'WeekOfYear', 'Day', 'Month', 'Year', 
            'HourOfDay_sin', 'HourOfDay_cos', 'DayOfWeek_sin', 'DayOfWeek_cos']

data = data[features]
# Generate lagged features for continuous features like price and energy generation
def create_lagged_features(data, features, lag_steps):
    data_with_lags = data.copy()
    for feature in features:
        for lag in lag_steps:
            data_with_lags[f'{feature}_lag_{lag}'] = data_with_lags[feature].shift(lag)
    return data_with_lags.dropna()

lag_steps = [1, 24, 168]  # Previous hour, day, and week lags
data_with_lags = create_lagged_features(data, ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load'], lag_steps)

# Scale and prepare sequences for LSTM training
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_with_lags), columns=data_with_lags.columns, index=data_with_lags.index)

# Define function to create sequences
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # All features for each sequence
        y.append(data.iloc[i+seq_length][target_column])  # Target at the next time step
    return np.array(X), np.array(y)

seq_length = 24
# train_size = int(len(data_scaled) * 0.8)
# train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

train_data, test_data = train_test_split_7(data_scaled)  # Assuming train_test_split_7 is predefined

X_train, y_train = create_sequences(train_data, seq_length, target_column='price')
X_test, y_test = create_sequences(test_data, seq_length, target_column='price')

# Define and train the LSTM model
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mae')
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
lstm_model = create_lstm_model(input_shape)

# Convert to float32 to avoid TensorFlow dtype issues
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')


lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Predict and evaluate
lstm_lag_preds = lstm_model.predict(X_test)

# Invert scaling for predictions and actual values
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
lstm_lag_preds_inverse = scaler.inverse_transform(
    np.concatenate((lstm_lag_preds, X_test[:, -1, 1:]), axis=1))[:, 0]

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
# Calculate performance metrics
mae = mean_absolute_error(y_test_inverse, lstm_lag_preds_inverse)
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_lag_preds_inverse))

smape_value_attention = smape(y_test_inverse, lstm_lag_preds_inverse)

# Print performance metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value_attention:.2f}")


Shape of train_data: (13319, 28)
Shape of test_data: (4033, 28)
Epoch 1/20


/Users/libin/anaconda3/envs/th/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


374/374 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0868 - val_loss: 0.0468
Epoch 2/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0525 - val_loss: 0.0360
Epoch 3/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0428 - val_loss: 0.0389
Epoch 4/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0386 - val_loss: 0.0386
Epoch 5/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0344 - val_loss: 0.0316
Epoch 6/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0326 - val_loss: 0.0345
Epoch 7/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0312 - val_loss: 0.0255
Epoch 8/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0302 - val_loss: 0.0288
Epoch 9/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0271 - val_loss: 0.0212
Epoch 10/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0264 - val_loss: 0.0265
Epoch 11/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0249 - val_loss: 0.0329
Epoch 12/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/ste

In [ ]:
# Check for NaNs in X_train and y_train
print("NaNs in X_train:", np.isnan(X_train).sum())
print("NaNs in y_train:", np.isnan(y_train).sum())

# Check for Inf values
print("Infs in X_train:", np.isinf(X_train).sum())
print("Infs in y_train:", np.isinf(y_train).sum())


In [27]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

# Now check for NaNs
print("NaNs in X_train:", np.isnan(X_train).sum())
print("NaNs in y_train:", np.isnan(y_train).sum())

NaNs in X_train: 0
NaNs in y_train: 0


## Lagged + Attention

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Multiply, Permute, RepeatVector, Activation, Flatten
import keras.backend as K

# Load your dataset (assuming it has been preprocessed)
data = load_data()
data['DayOfWeek'] = data.index.dayofweek              # Day of the week (0=Monday, 6=Sunday)
data['WeekOfYear'] = data.index.isocalendar().week    # ISO week number of the year
data['Day'] = data.index.day                          # Day of the month
data['Month'] = data.index.month                      # Month of the year
data['Year'] = data.index.year                        # Year
data['PeriodOfDay'] = data.index.hour                 # Hour of the day (0-23)
# Define features to use for lagging and target column
features_to_lag = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load']
target_column = 'price'

# Add cyclic transformations for HourOfDay and DayOfWeek
data['HourOfDay_sin'] = np.sin(2 * np.pi * data['PeriodOfDay'] / 24)
data['HourOfDay_cos'] = np.cos(2 * np.pi * data['PeriodOfDay'] / 24)

data['DayOfWeek_sin'] = np.sin(2 * np.pi * data['DayOfWeek'] / 7)
data['DayOfWeek_cos'] = np.cos(2 * np.pi * data['DayOfWeek'] / 7)

# Drop original cyclic features if you want to avoid redundancy
data.drop(['PeriodOfDay', 'DayOfWeek'], axis=1, inplace=True)

# Use only current values for 'WeekOfYear', 'Day', 'Month', 'Year' without lagging
features = ['price', 'wind_offshore', 'wind_onshore', 'solar', 'total_load', 
            'WeekOfYear', 'Day', 'Month', 'Year', 
            'HourOfDay_sin', 'HourOfDay_cos', 'DayOfWeek_sin', 'DayOfWeek_cos']
# data = data[features]
data = data[features]
# Generate lagged features
def create_lagged_features(data, features, lag_steps):
    data_with_lags = data.copy()
    for feature in features:
        for lag in lag_steps:
            data_with_lags[f'{feature}_lag_{lag}'] = data_with_lags[feature].shift(lag)
    return data_with_lags.dropna()  # Drop rows with NaN values from lagging

# Define lag intervals (e.g., previous 1 hour, 24 hours, and 168 hours)
lag_steps = [1, 24, 168]
data_with_lags = create_lagged_features(data, features_to_lag, lag_steps)

# Scale the data
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data_with_lags), columns=data_with_lags.columns, index=data_with_lags.index)

# Sequence Preparation Function
def create_sequences(data, seq_length, target_column):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length].values)  # Include all features within sequence length
        y.append(data.iloc[i+seq_length][target_column])  # Target at the next time step
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 24  # e.g., 24 hours or 1 day

# Split data into training and testing sets
# train_size = int(len(data_scaled) * 0.8)
# train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]
train_data, test_data = train_test_split_7(data_scaled)  # Assuming train_test_split_7 is predefined

# Create sequences for training and testing
X_train, y_train = create_sequences(train_data, seq_length, target_column)
X_test, y_test = create_sequences(test_data, seq_length, target_column)

# Attention Mechanism
def attention_3d_block(inputs):
    attention = Dense(1, activation='tanh')(inputs)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(inputs.shape[2])(attention)
    attention = Permute([2, 1])(attention)
    output_attention = Multiply()([inputs, attention])
    return output_attention

# Define LSTM model with Attention Mechanism
def create_lstm_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    lstm_out = LSTM(64, return_sequences=True)(inputs)
    lstm_out = Dropout(0.2)(lstm_out)
    
    # Apply attention mechanism
    attention_out = attention_3d_block(lstm_out)
    lstm_out_2 = LSTM(32, return_sequences=False)(attention_out)
    lstm_out_2 = Dropout(0.2)(lstm_out_2)
    
    outputs = Dense(1)(lstm_out_2)  # Output layer for regression
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='mae')
    return model

# Initialize and train the model
input_shape = (X_train.shape[1], X_train.shape[2])
lstm_attention_model = create_lstm_attention_model(input_shape)
lstm_attention_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# Predict on test data
lstm_attention_preds = lstm_attention_model.predict(X_test)

# Invert scaling for evaluation
y_test_inverse = scaler.inverse_transform(
    np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
lstm_attention_preds_inverse = scaler.inverse_transform(
    np.concatenate((lstm_attention_preds, X_test[:, -1, 1:]), axis=1))[:, 0]

# Calculate performance metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_inverse, lstm_attention_preds_inverse)
rmse = np.sqrt(mean_squared_error(y_test_inverse, lstm_attention_preds_inverse))
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


Shape of train_data: (13319, 28)
Shape of test_data: (4033, 28)
Epoch 1/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.1141 - val_loss: 0.0462
Epoch 2/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0567 - val_loss: 0.0394
Epoch 3/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0478 - val_loss: 0.0328
Epoch 4/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0436 - val_loss: 0.0405
Epoch 5/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0403 - val_loss: 0.0321
Epoch 6/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0378 - val_loss: 0.0330
Epoch 7/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0341 - val_loss: 0.0306
Epoch 8/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0335 - val_loss: 0.0274
Epoch 9/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0307 - val_loss: 0.0270
Epoch 10/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0293 - val_loss: 0.0244
Epoch 11/20
374/374 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.0

In [33]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
# smape_value = smape(y_test_inverse, lstm_preds_inverse)
smape_value_attention_lag = smape(y_test_inverse, lstm_attention_preds_inverse)

# Print performance metrics
print(f"Symmetric Mean Absolute Percentage Error (sMAPE): {smape_value_attention_lag:.2f}")

Symmetric Mean Absolute Percentage Error (sMAPE): 19.70
